In [1]:
import glob
import json
import openslide
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path0 = './Data/training'
plist = np.sort(glob.glob(path0+'/*.mrxs'))
json_data = {}

In [11]:
def bbox_position(img:np.array)->(int, int, int, int):
    '''
    The function that return bbox position by KITTI format.
    (x1, y1) - (x2, y2)
    '''
    
    for i in range(img.shape[0]):
        if img[i,:].sum() != 0:
            x1 = i
            break
            
    for i in range(img.shape[1]):
        if img[:,i].sum() != 0:
            y1 = i
            break
            
    for i in reversed(range(img.shape[0])):
        if img[i,:].sum() != 0:
            x2 = i
            break
        
    for i in reversed(range(img.shape[1])):
        if img[:,i].sum() != 0:
            y2 = i
            break
    
    
    return x1, y1, x2, y2

In [12]:
# Becuase of memory buffers, maximum N param is 1
n = 1

for pn in plist[2:]:
    pn_pid = pn.split('.mrxs')[0].split('/')[-1]
    pn_sl = openslide.OpenSlide(pn)
    pn_shape = pn_sl.level_dimensions[n]
    
    img = pn_sl.read_region((0,0),n,pn_shape).convert('L')
    img = np.array(img)
    
    x1, y1, x2, y2 = bbox_position(img)
    
    json_data[pn_pid] = {
        'lv' : n,
        'x1' : x1,
        'y1' : y1,
        'x2' : x2,
        'y2' : y2,
    }
    

In [14]:
with open('./LN_metastasis_bbox_position.json', 'w', encoding='utf-8') as file:
    json.dump(json_data, file)